# Lab 6

## Parte 1

In [35]:
import json
from pandas import json_normalize
from flare.data_science.features import domain_tld_extract
import pandas as pd

In [12]:
with open('large_eve.json', 'r') as f:
    data = [ json.loads(l) for l in f.readlines() if 'dns' in l]

assert data is not None

In [13]:
df = json_normalize(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21484 entries, 0 to 21483
Columns: 163 entries, timestamp to stats.http.memcap
dtypes: float64(129), object(34)
memory usage: 26.7+ MB


In [14]:
df.sample(2)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,stats.flow_mgr.rows_skipped,stats.flow_mgr.rows_empty,stats.flow_mgr.rows_busy,stats.flow_mgr.rows_maxlen,stats.file_store.open_files,stats.dns.memuse,stats.dns.memcap_state,stats.dns.memcap_global,stats.http.memuse,stats.http.memcap
15153,2017-07-22T19:29:19.014323-0500,1.038856e+15,3354669.0,dns,150.0,192.168.207.4,53.0,192.168.205.188,55640.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1464,2017-07-22T17:45:57.783232-0500,1.132581e+14,NaN,flow,NaN,192.168.205.58,33909.0,192.168.25.123,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df = df.loc[df['dns.rrtype'] == 'A']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 21303
Columns: 163 entries, timestamp to stats.http.memcap
dtypes: float64(129), object(34)
memory usage: 3.6+ MB


In [16]:
df = df.drop_duplicates(subset=['dns.rrname'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177 entries, 0 to 21297
Columns: 163 entries, timestamp to stats.http.memcap
dtypes: float64(129), object(34)
memory usage: 226.8+ KB


In [17]:
df['domain_tld'] = df['dns.rrname'].apply(domain_tld_extract)

In [18]:
df['domain_tld'].sample(5)

1705       acunetix.com
893      postgresql.org
1880      microsoft.com
1836         flickr.com
496     stopbadware.org
Name: domain_tld, dtype: object

## Parte 2

In [19]:
import pickle

In [20]:
dtc = pickle.load(open('ht1model.m', 'rb'))

In [36]:
new_df = pd.DataFrame()

In [51]:
new_df['ip'] = df['dest_ip'].apply(lambda s: 1 if s is not None else 0 )
for missing_col in ['nb_hyperlinks','domain_in_title','domain_age','google_index','page_rank', 'phish_hints']:
    new_df[missing_col] = 0

In [52]:
contains = lambda c: lambda s: 1 if c in s else 0 
nb_www = contains('www.')

new_df['nb_www'] = df['dns.rrname'].apply(nb_www)

In [53]:
digits_ratio = lambda s: sum(c.isdigit() for c in s) / max(sum(not c.isdigit() for c in s), 1)
new_df['digit_ratio_url'] = df['dns.rrname'].apply(digits_ratio)
new_df['digit_ratio_hostname'] = df['dns.rrname'].apply(digits_ratio)

In [58]:
res = dtc.predict(new_df)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [60]:
df[[True if s else False for s in res]]['dns.rrname']

Series([], Name: dns.rrname, dtype: object)

In [61]:
from flare.tools.umbrella import Umbrella

In [64]:
umb = Umbrella()

[+] Updated Umbrella Top 1 Million list...


In [66]:
df['in_umbrella'] = df['dns.rrname'].apply(lambda s: 1 if umb.domain_tld_in_umbrella(s) else 0)

In [69]:
df.loc[df['in_umbrella'] == 1]

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,stats.flow_mgr.rows_busy,stats.flow_mgr.rows_maxlen,stats.file_store.open_files,stats.dns.memuse,stats.dns.memcap_state,stats.dns.memcap_global,stats.http.memuse,stats.http.memcap,domain_tld,in_umbrella
468,2017-07-22T17:40:48.114275-0500,1.014351e+15,96677.0,dns,140.0,192.168.204.63,52070.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,portswigger.net,1
973,2017-07-22T17:48:23.255644-0500,1.964866e+15,203832.0,dns,120.0,192.168.202.172,54399.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sourceforge.net,1
1599,2017-07-22T17:53:44.404783-0500,1.151274e+15,234696.0,dns,120.0,192.168.202.172,55750.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,freepbx.org,1
2879,2017-07-22T18:24:52.733891-0500,9.570046e+14,482149.0,dns,140.0,192.168.204.60,62393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,google.com,1


El modelo del laboratorio 1 no detecto ningun dominio sospechoso. 